In [ ]:
from alpha_vantage.timeseries import TimeSeries
from mpl_finance import candlestick2_ohlc
from tensorflow import keras
import shap

In [1]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import matplotlib.pyplot as plt
from mpl_finance import candlestick2_ohlc
import random
import os
import time

C:\ProgramData\Anaconda3\lib\site-packages\mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


In [2]:
def empty_folders():
    
    folder_path = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/buy'
    for file_object in os.listdir(folder_path):
        file_object_path = os.path.join(folder_path, file_object)
        os.unlink(file_object_path)
        
    folder_path = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/sell'
    for file_object in os.listdir(folder_path):
        file_object_path = os.path.join(folder_path, file_object)
        os.unlink(file_object_path)
        
    folder_path = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/buy'
    for file_object in os.listdir(folder_path):
        file_object_path = os.path.join(folder_path, file_object)
        os.unlink(file_object_path)
        
    folder_path = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/sell'
    for file_object in os.listdir(folder_path):
        file_object_path = os.path.join(folder_path, file_object)
        os.unlink(file_object_path)

In [3]:
def save_images(ticker,interval):
    
    ts = TimeSeries(output_format='pandas')
    
    if interval == '15min' or interval == '30min' or interval == '60min':
        
        data, meta_data = ts.get_intraday(symbol = ticker, interval = interval, outputsize = 'full')
        
    if interval == 'daily':
        
        data, meta_data = ts.get_daily(symbol = ticker, outputsize = 'full')
        
    if interval == 'weekly':
        
        data, meta_data = ts.get_weekly(symbol = ticker)
        
    if interval == 'monthly':
        
        data, meta_data = ts.get_monthly(symbol = ticker)
        
    data = data[['1. open', '2. high', '3. low', '4. close']]
    data.columns = ['open', 'high', 'low', 'close']
    data = data.iloc[::-1]
    
    numero_de_candles = 15
    numero_de_periodos_investido = 5

    numero_de_imagens = int((len(data) - numero_de_periodos_investido - 1)/numero_de_candles)

    fig, ax = plt.subplots()

    for i in range(numero_de_imagens):

        start = i*numero_de_candles
        end = (i + 1)*numero_de_candles
        periodo = data.iloc[start:end]

        args = periodo.values.T

        candlestick2_ohlc(ax, *args, width=0.5, colorup='green', colordown='red', alpha=1)
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)
        ax.axis("off")

        
        title = ticker + ' ' + interval + ' from ' + str(periodo.index[0])[:10] + ' to ' + str(periodo.index[-1])[:10]
        
        first_price = periodo['close'][0]
        last_price = data['close'][end + numero_de_periodos_investido]
        
        folder = str(random.choices(
            population = ['Treinamento/', 'Teste/'],
            weights = [0.7, 0.3]
        ))[2:-2]
        
        if (first_price / last_price) < 2 and (last_price / first_price) < 2:

            if (data['close'][end + numero_de_periodos_investido] > 1.003*periodo['close'][-1]  
            and title.replace(" ", "_").replace(":", ".") + '.png' not in os.listdir(folder + 'buy')):

                fig.savefig(folder + 'buy/' + title.replace(" ", "_").replace(":", ".") + '.png')

            elif (data['close'][end + numero_de_periodos_investido] < 0.997*periodo['close'][-1] 
            and title.replace(" ", "_").replace(":", ".") + '.png' not in os.listdir(folder + 'sell')):

                fig.savefig(folder + 'sell/' + title.replace(" ", "_").replace(":", ".") + '.png')

        plt.cla()

    plt.close(fig)

In [4]:
def corrigir_base():
    
    len_train_sell = len(os.listdir('C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/sell'))
    len_train_buy = len(os.listdir('C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/buy'))
    len_test_sell = len(os.listdir('C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/sell'))
    len_test_buy = len(os.listdir('C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/buy'))
    
    if len_train_buy > len_train_sell:
        
        path = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/buy'
        files = os.listdir(path)
        for file in random.sample(files,(len_train_buy - len_train_sell)):
            os.remove(path+'/'+file)
            
    elif len_train_buy < len_train_sell:
        
        path = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/sell'
        files = os.listdir(path)
        for file in random.sample(files,(len_train_sell - len_train_buy)):
            os.remove(path+'/'+file)
            
            
    if len_test_buy > len_test_sell:
        
        path = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/buy'
        files = os.listdir(path)
        for file in random.sample(files,(len_test_buy - len_test_sell)):
            os.remove(path+'/'+file)
            
    elif len_test_buy < len_test_sell:
        
        path = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/sell'
        files = os.listdir(path)
        for file in random.sample(files,(len_test_sell - len_test_buy)):
            os.remove(path+'/'+file)
    

In [5]:
def Data_base(n):
    
    #empty_folders()
    my_dpi = 72
    plt.rcParams["figure.figsize"] = (500/my_dpi,500/my_dpi)

    len_sell = 0
    len_buy = 0
    target_len = n

    while len_sell < target_len/2 or len_buy < target_len/2:

        time.sleep(12)

        interval = str(random.choices(
            population = ['monthly', 'weekly', 'daily', '60min', '30min', '15min'],
            weights = [7.75179438, 2.00326963, 0.2199004 , 2.96476033, 2.27640876, 1.62488321]
        ))[2:-2]

        ticker = str(random.choices(pd.read_csv('C:/Users/vitor/Desktop/Python Stuff/Python Programs/Mercado Financeiro\
/Otimização de Carteira/sp500_constituents.csv')['Symbol']))[2:-2]
        
        try:

            save_images(ticker, interval)
            
        except:
            
            continue

        len_train_sell = len(os.listdir('C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/sell'))
        len_train_buy = len(os.listdir('C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/buy'))
        len_test_sell = len(os.listdir('C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/sell'))
        len_test_buy = len(os.listdir('C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/buy'))
        len_sell = len_train_sell + len_test_sell
        len_buy = len_train_buy + len_test_buy
        total_len = len_sell + len_buy
        
        print(len_sell)
        
    corrigir_base()
        
    return str(total_len)

In [6]:
Data_base(30000)

9974
9977
9977
9977
9985
9994
9999
10000
10004
10009
10011
10023
10033
10034
10045
10048
10053
10054
10062
10062
10070
10070
10073
10088
10094
10250
10250
10264
10265
10266
10283
10285
10288
10294
10295
10301
10307
10310
10334
10337
10339
10340
10347
10356
10362
10366
10374
10375
10376
10379
10401
10408
10413
10428
10436
10443
10450
10460
10462
10465
10470
10483
10492
10497
10501
10501
10503
10505
10510
10523
10540
10548
10559
10567
10571
10584
10587
10597
10604
10605
10610
10612
10618
10626
10650
10651
10659
10683
10687
10691
10703
10706
10717
10721
10729
10741
10748
10749
10756
10775
10779
10781
10781
10792
10809
10823
10825
10831
10833
10833
10845
10847
10847
10861
10879
10884
10884
10895
10922
10933
10940
10964
10965
10989
10996
10999
11005
11009
11014
11022
11024
11025
11032


KeyboardInterrupt: 

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x0000028B872C4D38> (for post_execute):


IndexError: list index out of range

IndexError: list index out of range

<Figure size 500x500 with 1 Axes>

In [24]:
corrigir_base()

In [18]:
import shutil

a = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/up'
b = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/buy'
c = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/buy'
e = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Teste/sell'
f = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Data Base/Treinamento/sell'

for i in os.listdir(a):
    
    d = str(random.choices(
            population = [c, b],
            weights = [0.7, 0.3]
        ))[2:-2]
    
    shutil.move(str(a +'/'+ str(i)), d)
    


In [29]:
test_buy = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Model/Test/buy'
train_buy = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Model/Train/buy'
test_sell = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Model/Test/sell'
train_sell = 'C:/Users/vitor/Desktop/Education/Graduação/LAMFO/#img_candles/Model/Train/sell'

In [38]:
len(os.listdir(test_buy))

9672